In [1]:
import numpy as np
import pandas as pd
import uproot
import csv
import os
import glob

#### Choose file to work with:

In [86]:
def get_file_name():
    
    file_name = input("\n Which .root file do you want to open?\n")
    if file_name is not '' and file_name.endswith('.root'):
        if os.path.isfile(file_name):
            return file_name
        else:
            print('\n No such file... Try again. \n')
            get_file_name()
    else:
        print ("\n No or wrong input given... Try again. \n")
        get_file_name()

#### Get the names of all the trees in the file

In [53]:
def get_tree_names(f_name):
    
    try:
        f = uproot.open(f_name)
        trees = f.keys()
        tree_names=[]

        print("\n Tree names successfully stored. They are: \n")
        for tree in trees:
            tree = str(tree)
            tree = tree[2:len(tree)-3]
            tree_names.append(tree)
            print(tree)
            print('\n')
            
        return tree_names

    except:
        print ("\n ERROR: Trees not found. Check your file before continuing...\n")
    
    


### Srotolo i tree e faccio tre singoli file 

In [63]:
def root_tree_to_csv():

    fname = get_file_name()
    files_out= []

#    sgn_treename = 'signal_bbA_MA300tree'
#    bkgttbar_treename = 'bkg_ttbar_nlotree'
#    bkgDY_treename = 'bkg_DY_nlo1tree'
    
    woods = get_tree_names(fname)
    
    #print (woods)

    for tree in woods:
        print ("Reading tree {} in file {}".format(tree, fname))
        data = uproot.open(fname)[tree]
        names = data.keys()

        #data.arrays(names)
        out = pd.DataFrame.from_dict(data.arrays(names), dtype= str)
        out_file_name = tree + '.csv'
        out.to_csv(out_file_name)
        files_out.append(out_file_name)
        print ('created csv file ' + outfilename)

#### Aggiungo la colonna con label

In [76]:
def add_label_column():
    try:
        files = [f for f in glob.glob('*.{}'.format('csv'))]
        for file in files:
            reader = csv.reader(open(file, 'r'))
            writer = csv.writer(open("l_" + file, 'w'))

            headers = next(reader)
            headers.append("label")

            writer.writerow(headers)
            for row in reader:
                if file == "signal_bbA_MA300tree.csv":
                    row.append("signal")
                    writer.writerow(row)
                else:
                    row.append("background")
                    writer.writerow(row)
    except:
        print('\n ERROR: No csv file found... \n')

### Unisco i tre file

In [85]:
def file_merger():

    all_filenames = [f for f in glob.glob('*.{}'.format('csv'))]
    #combine all files in the list
    combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ], sort = False)
    #pick a name for the new file
    file_out_name = input('\n How would you like to name the new file? \n')
    #export to csv
    combined_csv.to_csv( file_out_name, index=False, encoding='utf-8-sig')

In [ ]:
root_tree_to_csv()

In [73]:
add_label_column()

In [ ]:
file_merger()